# Lista 07 - Comparando Classificadores

# Exercício 01:

Analise o desempenho do kNN e de uma Regressão Logística Regularizada para **pelo menos um** dos conjuntos de dados abaixo:

* [Avaliação de carros](http://archive.ics.uci.edu/ml/datasets/Car+Evaluation)
* [Avaliação de vinhos](http://archive.ics.uci.edu/ml/datasets/Wine+Quality)
* [Resultados de partidas do jogo Dota](http://archive.ics.uci.edu/ml/datasets/Dota2+Games+Results) (desafiador!)

Para a questão, faça as seguintes tarefas:

* Realize treino, validação e teste
* Compare as métricas no teste
* Reporte a precisão, revocação, F1 e a matriz de confusão

Como já estamos no fim da matéria, você pode agora fazer uso da biblioteca scikit-learn. Afinal, no dia a dia, não implementamos tudo do zero. Abaixo temos os imports que vocês precisam. Leiam a API da biblioteca para saber como fazer uso da mesma.

In [25]:
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from sklearn.neighbors import KNeighborsClassifier

# Um fator importante é que o SKLearn não cria conjuntos de validação para você. Você tem algumas abordagens,
# uma é realizar um novo split no treino. Outra é fazer uso de classificadores com CV no fim.
# Tipo LogisticRegressionCV (ver na API). Por fim, você pode fazer uso da classe GridSearchCV.
# Leia a documentação da mesma.
from sklearn.model_selection import train_test_split

In [127]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import hmean
from scipy.stats.mstats import gmean

## Base de Dados

In [11]:
# dfTrain = pd.read_csv("dota2Train.csv", header=None)
# dfTest = pd.read_csv("dota2Test.csv", header=None)
# columns = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "ph", "sulphates", "alcohol", "quality"]
df = pd.read_csv("winequality-red.csv", sep=";")

In [33]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Qualidade do vinho

Segundo a descrição dos dados, a qualidade do vinho é um valor entre 0 e 10, mas apenas os valores abaixo são observados.

In [71]:
quality_labels = np.sort(df["quality"].unique())
quality_labels

array([3, 4, 5, 6, 7, 8])

## Separação treino-teste

O conjunto de dados foi separado em treino e teste com proporção de 7:3.

In [56]:
X = df.values[:, :-1]
y = df.values[:, -1]
y_true = y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Regressão logística

Usando a função de regressão logística já com validação cruzada.

Todas as informações sobre o vinho foram utilizadas para prever a qualidade. O otimizador padrão da classe `LogisticRegressionCV` do `sklearn` é o [algoritmo de Broyden–Fletcher–Goldfarb–Shanno](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm). Com 100 e com 1000, o algoritmo não conseguiu convergir. Para este conjunto de dados foram feitas 10000 iterações para cada teste e paralelizado em 2 tarefas.

In [64]:
logistic_regression_cv = LogisticRegressionCV(
    cv=5, max_iter=10000, n_jobs=2, verbose=1, multi_class="auto").fit(
    X_train, y_train)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:   42.0s finished


In [67]:
y_pred = logistic_regression_cv.predict(X_test)

### Matriz de confusão

In [116]:
pd.DataFrame(
    confusion_matrix(y_test, y_pred, labels=quality_labels), index=quality_labels, columns=quality_labels)

,3,4,5,6,7,8
3,0,2,2,1,0,0
4,0,0,12,5,1,0
5,0,0,150,49,1,0
6,0,0,69,110,13,0
7,0,0,4,43,15,0
8,0,0,0,0,3,0


### Previsão, revocação e F1

In [122]:
prfs = precision_recall_fscore_support(y_test, y_pred, labels=quality_labels)
pd.DataFrame(prfs[:-1], columns=quality_labels, index=["Precisão", "Revocação", "F1"])

,3,4,5,6,7,8
Precisão,0.0,0.0,0.632911,0.528846,0.454545,0.0
Revocação,0.0,0.0,0.750000,0.572917,0.241935,0.0
F1,0.0,0.0,0.686499,0.550000,0.315789,0.0


Há valores de qualidade no conjunto de teste que não foram preditos.

In [82]:
set(y_test) - set(y_pred)

{3.0, 8.0}

## K-Nearest Neighbors

No `sklearn` não há classe para o KNN com validação cruzada. Para isto, a classe `GridSearchCV` é utilizada para procurar o melhor número de vizinhos. Utilizaremos um intervalo de 1 até 50 vizinhos para validação.

Referência: https://towardsdatascience.com/building-a-k-nearest-neighbors-k-nn-model-with-scikit-learn-51209555453a

In [104]:
knn = KNeighborsClassifier()
param_grid = {"n_neighbors": np.arange(1, 51)}
knn_gscv = GridSearchCV(knn, param_grid, n_jobs=2, cv=5, verbose=1).fit(X_test, y_test)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/opt/anaconda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    2.7s finished
/opt/anaconda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [107]:
knn_gscv.best_params_

{'n_neighbors': 36}

In [114]:
y_pred_knn = knn_gscv.predict(X_test)

### Matriz de confusão

In [117]:
pd.DataFrame(
    confusion_matrix(y_test, y_pred_knn, labels=quality_labels), index=quality_labels, columns=quality_labels)

,3,4,5,6,7,8
3,0,0,3,2,0,0
4,0,0,11,7,0,0
5,0,0,130,70,0,0
6,0,0,68,122,2,0
7,0,0,21,39,2,0
8,0,0,1,2,0,0


### Previsão, revocação e F1

In [120]:
prfs_knn = precision_recall_fscore_support(y_test, y_pred_knn, labels=quality_labels)
pd.DataFrame(prfs[:-1], columns=quality_labels, index=["Precisão", "Revocação", "F1"])

,3,4,5,6,7,8
Precisão,0.0,0.0,0.555556,0.504132,0.500000,0.0
Revocação,0.0,0.0,0.650000,0.635417,0.032258,0.0
F1,0.0,0.0,0.599078,0.562212,0.060606,0.0


Explique e discuta sobre os resultados encontrados no campo abaixo.

Os dois algoritmos tiveram valores diferentes para F1. Para julgar qual é o melhor, eu proponho fazer uma média entre os valores de F1. Temos então:

In [131]:
# (Regressão Logística, KNN)
prfs[2].mean(), prfs_knn[2].mean()

(0.2587147215865751, 0.20364939719778427)

Se o uso da média para comparar estiver correto, pode-se afirmar que a regressão logística funcionou melhor para este caso.